In [37]:
#  https://cyc1am3n.github.io/2018/11/10/classifying_korean_movie_review.html
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [38]:
df = pd.read_csv('C:/ratings_train.txt', sep='\t')
df = df.drop(columns='id')
df.shape

(150000, 2)

In [39]:
df.sample(2)

,document,label
21151,스카이라인진짜 존잼인데 무슨소리 ㅡㅡ 댓글 얼척없네 외계인다룬영화중에 가장재미있게본...,1
38780,로그인하게 만드네 왤케 평점이 낮음?? 재밋어여ㅋㅋ,1


In [40]:
df.dtypes, df.isna().sum().sum()

(document    object
 label        int64
 dtype: object, 5)

In [41]:
df = df.dropna() # 누락행 제거

### 어떻게 자연어를 숫자로 바꿀까?

In [42]:
corpus = ['I am korean', 'You are Korean']

In [43]:
from sklearn.feature_extraction.text import CountVectorizer # 갯수를 세고 백터로 저장

In [44]:
cv = CountVectorizer().fit(corpus) #데이터 전부 전달

In [45]:
cv.vocabulary_ # 어휘 사전

{'am': 0, 'are': 1, 'korean': 2, 'you': 3}

In [46]:
vector = cv.transform(corpus) # 변환
vector

<2x4 sparse matrix of type '<class 'numpy.int64'>'
	with 5 stored elements in Compressed Sparse Row format>

In [47]:
vector.toarray() # 회소 행렬을 Dense행렬(평소에 사용하는 행렬)로 변환

array([[1, 0, 1, 0],
       [0, 1, 1, 1]], dtype=int64)

### 실전

In [75]:
df = pd.read_csv('C:/ratings_train.txt', sep='\t')
df = df.drop(columns='id')
df.shape
df = df.iloc[:1000] # 속도를 위해서 100개로 제한해둡니다.

In [76]:
df.head(5)

,document,label
0,아 더빙.. 진짜 짜증나네요 목소리,0
1,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,너무재밓었다그래서보는것을추천한다,0
3,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [77]:
corpus = df['document'].values
len(corpus)

1000

In [78]:
cv = CountVectorizer().fit(corpus) # 데이터 전부 전달
len(cv.vocabulary_)
# cv.vocabulary_ #어휘 사전

5274

In [79]:
vector = cv.transform(corpus)
vector

<1000x5274 sparse matrix of type '<class 'numpy.int64'>'
	with 7245 stored elements in Compressed Sparse Row format>

In [80]:
vector.toarray()[0].sum(), vector.toarray()[0]

(4, array([0, 0, 0, ..., 0, 0, 0], dtype=int64))

# KoNLPy를 이용한 한국어 영화 리뷰 감정 분석

In [81]:
from konlpy.tag import Okt

In [82]:
okt = Okt()
okt.morphs('단독입찰보다 복수입찰의 경우')

['단독', '입찰', '보다', '복수', '입찰', '의', '경우']

In [83]:
cv = CountVectorizer(tokenizer=okt.morphs).fit(corpus) # 데이터 전부 전달
# 잘라진 단어를 토큰이라고 합니다.
# 문장을 자르는 것을 토크닝이라고 하고, 문장을 자르는 개체를 토큰나이저라고 합니다.
len(cv.vocabulary_)
#cv.vocabulary_

4767

In [84]:
df.head()

,document,label
0,아 더빙.. 진짜 짜증나네요 목소리,0
1,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,너무재밓었다그래서보는것을추천한다,0
3,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [85]:
y = df.label.values
x = cv.transform(corpus)

In [86]:
from sklearn.linear_model import LinearRegression
model = LinearRegression().fit(x,y)
model.score(x,y)

0.9999999999912365

In [90]:
test = ['아 영화 재미없다.', '별로였어요','유익해요','신나고 긴장됨']
model.predict(cv.transform(test))

array([-0.86024764,  0.06223691,  0.492     ,  0.51303614])

In [92]:
model.coef_

array([ 0.11112086,  0.0038683 ,  0.02314049, ...,  0.11881577,
        0.12509583, -0.06298543])

In [1]:
i = 0
for key in cv.vocabulary_.keys():
    c = (model.coef_[0,i])
    if c > -1.0:
        print(key, cv.vocabulary_[key])
    i += 1
    

NameError: name 'cv' is not defined